In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
from nltk.corpus import stopwords

Using TensorFlow backend.


In [ ]:
import pandas as pd
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

# Drop the columns that are not required for the neural network.
del train_df['qid']

In [ ]:
X = train_df['question_text'].values
Y = train_df['target'].values

from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y,test_size = 0.1)

train_df = pd.DataFrame(data={"question_text":X_train, "target":Y_train})
valid_df = pd.DataFrame(data={"question_text":X_valid, "target":Y_valid})

In [ ]:
# Class count
count_class_0, count_class_1 = train_df.target.value_counts()

# Divide by class
df_class_0 = train_df[train_df['target'] == 0]
df_class_1 = train_df[train_df['target'] == 1]
df_class_0_under = df_class_0.sample(count_class_1)
train_df = pd.concat([df_class_0_under, df_class_1], axis=0)
print('Random under-sampling:')
print(train_df.target.value_counts())
train_df.target.value_counts().plot(kind='bar', title='Count (target)');

In [3]:
import string
getPunctuations = str.maketrans("","",string.punctuation) 

from nltk.corpus import stopwords
stop = stopwords.words("english")
train_df['question_text'] = train_df['question_text'].apply(lambda x: x.lower())
train_df['question_text'] = train_df['question_text'].apply(lambda x: x.translate(getPunctuations))
train_df['question_text'] = train_df['question_text'].apply(lambda x: x.split())
train_df['question_text'] = train_df['question_text'].apply(lambda x: [item for item in x if item not in stop])
train_df.head()

NameError: name 'train_df' is not defined

In [ ]:
valid_df['question_text'] = valid_df['question_text'].apply(lambda x: x.lower())
valid_df['question_text'] = valid_df['question_text'].apply(lambda x: x.translate(getPunctuations))
valid_df['question_text'] = valid_df['question_text'].apply(lambda x: x.split())
valid_df['question_text'] = valid_df['question_text'].apply(lambda x: [item for item in x if item not in stop])
valid_df.head()

In [ ]:
test_df['question_text'] = test_df['question_text'].apply(lambda x: x.lower())
test_df['question_text'] = test_df['question_text'].apply(lambda x: x.translate(getPunctuations))
test_df['question_text'] = test_df['question_text'].apply(lambda x: x.split())
test_df['question_text'] = test_df['question_text'].apply(lambda x: [item for item in x if item not in stop])
test_df.head()

In [ ]:
X_train = train_df['question_text'].values
Y_train = train_df['target'].values

X_valid = valid_df['question_text'].values
Y_valid = valid_df['target'].values

X_test = test_df['question_text'].values

In [ ]:
max_words = 1000
max_len = 200
tok = Tokenizer()
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
sequences_matrix.shape

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(sequences_matrix.max()+1,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
from keras import metrics
model = RNN()
# model.(Embedding(voc_size, 32))
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=5,validation_split=0.2,
    callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.001)])

In [ ]:
valid_sequences = tok.texts_to_sequences(X_valid)
valid_sequences_matrix = sequence.pad_sequences(valid_sequences,maxlen=max_len)

In [ ]:
valid_pred = model.predict(valid_sequences_matrix)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
score =0.0;
limit = 0.5;
for thresh in np.arange(0.1, 0.99, 0.01):
    thresh = np.round(thresh, 2)
    print (confusion_matrix((valid_pred > thresh).astype(int),Y_valid))
    f1 = f1_score((valid_pred > thresh).astype(int),Y_valid)
    print('F1 score at threshold {} is {}'.format(thresh,f1))
    if f1>score:
        limit = thresh;
        score = f1

In [ ]:
print(score,limit)

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
pred = model.predict(test_sequences_matrix)
Y_out = (pd.DataFrame((pred>limit).astype(int))[0]).values
submission = pd.DataFrame(data={"qid":test_df["qid"], "prediction":Y_out})
submission.to_csv( "submission.csv", index=False)

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
score =0.0;
limit = 0.5;
for thresh in np.arange(0.1, 0.99, 0.01):
    thresh = np.round(thresh, 2)
    print confusion_matrix((pred > thresh).astype(int),Y_test)
    f1 = f1_score((pred > thresh).astype(int),Y_test)
    print('F1 score at threshold {} is {}'.format(thresh,f1))
    if f1>score:
        limit = thresh;
        score = f1
        
Y_out = (pd.DataFrame(np.around(pred).tolist())[0].astype(str).astype(float).astype(int)).values
